In [1]:
import numpy as np
import pandas as pd
import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [2]:
def save_varible_pickle(var, name, model = False):
    filename = name + '.pickle'
    if model:
        joblib.dump(var, filename = filename)
    else:  
        with open(filename, 'wb') as f:
            pickle.dump(var, f, pickle.HIGHEST_PROTOCOL)

def load_variable_pickle(name, model = False):
    filename = name
    if model:
        return joblib.load(filename = filename)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)
        

In [3]:
def choose_system(os):
    if os == 'mac':
        train_path = '/Users/yichen/Desktop/toxic/input/train.csv'
        test_path = '/Users/yichen/Desktop/toxic/input/test.csv'
    elif os == 'linux':
        train_path = '/home/yiche/Desktop/toxic/input/train.csv'
        test_path = '/home/yiche/Desktop/toxic/input/test.csv'
    elif os == 'aws':
        train_path = 'train.csv'
        test_path = 'test.csv'
    
    train = pd.read_csv(train_path).fillna(' ')
    test = pd.read_csv(test_path).fillna(' ')
    return train, test
    
    

In [4]:
train, test = choose_system('linux')
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
test.shape

(153164, 2)

In [5]:
repl = {
    "&lt;3": " good ", ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

In [6]:
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [7]:
all_text.shape

(312735,)

In [9]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    min_df = 2,
    stop_words = 'english',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 3),
    max_features=30000
    )
    

char_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='char',
    min_df = 2,
    ngram_range=(2, 6),
    max_features=30000)

vectorizer = make_union(word_vectorizer, char_vectorizer)

In [10]:
vectorizer.fit(all_text)
train_features = vectorizer.transform(train_text)
train_features.shape

/Users/yichen/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(159571, 60000)

In [11]:
test_features = vectorizer.transform(test_text)

/Users/yichen/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [13]:
test_features.shape

(153164, 60000)

In [43]:
import pickle
from sklearn.externals import joblib
save_varible_pickle(train_features, train_features_save)
save_varible_pickle(test_features, test_features_save)
save_varible_pickle(clf, _logistic, model = True)

NameError: name 'train_features_save' is not defined

In [8]:
import pickle
from sklearn.externals import joblib
test_features = load_variable_pickle('test_features_save.pickle')
train_features = load_variable_pickle('train_features_save.pickle')
clf = load_variable_pickle('_logistic_model5.pickle')

/home/yiche/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
print(test_features.shape, train_features.shape)


(153164, 60000) (159571, 60000)


In [16]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [11]:
train_target = train['toxic']
X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size = 0.1, random_state = 1023)
print('X_train shape: {}, y_train shape: {}\
        X_test shape:{}, y_test shape:{} '.format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

X_train shape: (143613, 60000), y_train shape: (143613,)        X_test shape:(15958, 60000), y_test shape:(15958,) 


In [12]:
from sklearn.linear_model import LogisticRegression
logistic_lvl1 = LogisticRegression(solver='sag')


logistic_lvl1.fit(X_train, y_train)

log_pred = logistic_lvl1.predict(X_test)st, lo)
log_test_pred = logistic_lvl1.predict(test_features)

KeyboardInterrupt: 

In [17]:
from sklearn.svm import LinearSVC
svc_lvl1 = LinearSVC(random_state = 1023)
svc_lvl1.fit(X_train, y_train)
svc_pred = svc_lvl1.predict(X_test)
svc_test_pred = svc_lvl1.predict(test_features)

0.96283995488156415

In [88]:
scores_auc = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    print(train_target.shape)
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(
        classifier, train_features, train_target, cv=3, scoring='neg_log_loss'))
    scores_auc.append(np.abs(cv_score))
    print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    save_varible_pickle(clf, _logistic, model = True)
print('Total auc CV score is {}'.format(np.mean(scores_auc)))

(159571,)


SystemError: Bad call flags in PyCFunction_Call. METH_OLDARGS is no longer supported!

In [18]:
submission.to_csv('submission.csv', index=False)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier
scores_rfc = []
submission_rfc = pd.DataFrame.from_dict({'id': test['id']})
train_target = []
couter = 0
for class_name in class_names:
    train_target = train[class_name]
    rfc_clf = RandomForestClassifier(n_estimators= 200, random_state=11)
 
    #cv_score = np.mean(cross_val_score(
        #rfc_clf, train_features, train_target, cv=3, scoring='neg_log_loss'))
    #scores_rfc.append(cv_score)
    #print('auc CV score for class {} is {}'.format(class_name, cv_score))
    filaname = '_rfc_part' + str(couter)
    save_varible_pickle(rfc_clf, filaname, model = True)
    rfc_clf.fit(train_features, train_target)
    submission_rfc[class_name] = rfc_clt.predict_proba(test_features)[:, 1]
    couter = couter + 1

#print('Total auc CV score is {}'.format(np.mean(scores_rfc)))

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
rfc1 = GradientBoostingClassifier()
train_target = train['toxic']
cv_score = np.mean(cross_val_score(rfc1, train_features, train_target, cv = 5, scoring='roc_auc'))
print('cv score {}'.format(cv_score))